# Stochastic Fairness Python script

## Author: Samuel Rodriguez-Gonzalez, PhD
### Last modified: 2025-08-01


In [23]:
from gurobipy import *
import numpy as np
import random
import pandas as pd
from math import log
import winsound
import tdINDP_parameters_build

In [24]:
SVI = {
'Gas':
{
'Gas_node_3':[0.208,0.4504],
'Gas_node_4':[0.208,0.4504],
'Gas_node_5':[0.4505,0.6909],
'Gas_node_6':[0.208,0.4504],
'Gas_node_7':[0,0.2079],
'Gas_node_8':[0,0.2079],
'Gas_node_9':[0,0.9993],
'Gas_node_10':[0,0.2079],
'Gas_node_11':[0,0.2079],
'Gas_node_12':[0,0.2079],
'Gas_node_13':[0.4505,0.6909],
'Gas_node_14':[0.4505,0.6909],
'Gas_node_15':[0,0.2079]
},
'Power':
{
'Power_node_9':[0.691,0.8594],
'Power_node_10':[0.8595,0.9993],
'Power_node_11':[0,0.2079],
'Power_node_12':[0.691,0.8594],
'Power_node_13':[0,0.2079],
'Power_node_14':[0,0.9993],
'Power_node_15':[0.8595,0.9993],
'Power_node_16':[0.691,0.8594],
'Power_node_17':[0,0.9993],
'Power_node_18':[0.8595,0.9993],
'Power_node_19':[0,0.9993],
'Power_node_20':[0.208,0.4504],
'Power_node_21':[0.8595,0.9993],
'Power_node_22':[0.691,0.8594],
'Power_node_23':[0.8595,0.9993],
'Power_node_24':[0.691,0.8594],
'Power_node_25':[0,0.4504],
'Power_node_26':[0.691,0.8594],
'Power_node_27':[0.8595,0.9993],
'Power_node_28':[0.691,0.9993],
'Power_node_29':[0.691,0.8594],
'Power_node_30':[0.8595,0.9993],
'Power_node_31':[0,0.6909],
'Power_node_32':[0.208,0.6909],
'Power_node_33':[0,0.6909],
'Power_node_34':[0,0.9993],
'Power_node_35':[0.691,0.8594],
'Power_node_36':[0,0.9993],
'Power_node_37':[0,0.2079],
'Power_node_38':[0.8595,0.9993],
'Power_node_39':[0.208,0.4504],
'Power_node_40':[0.8595,0.9993],
'Power_node_41':[0.208,0.4504],
'Power_node_42':[0.208,0.4504],
'Power_node_43':[0.208,0.4504],
'Power_node_44':[0.208,0.4504],
'Power_node_45':[0.8595,0.9993],
'Power_node_46':[0,0.2079],
'Power_node_47':[0,0.2079],
'Power_node_48':[0.208,0.4504],
'Power_node_49':[0,0.4504],
'Power_node_50':[0,0.2079]
},
'Water':
{
'Water_node_15':[0.4505,0.6909],
'Water_node_16':[0,0.2079],
'Water_node_17':[0.8595,0.9993],
'Water_node_18':[0,0.9993],
'Water_node_19':[0.8595,0.9993],
'Water_node_20':[0,0.9993],
'Water_node_21':[0.8595,0.9993],
'Water_node_22':[0.208,0.9993],
'Water_node_23':[0,0.2079],
'Water_node_24':[0.4505,0.6909],
'Water_node_25':[0,0.2079],
'Water_node_26':[0.691,0.8594],
'Water_node_27':[0,0.4504],
'Water_node_28':[0.8595,0.9993],
'Water_node_29':[0.208,0.4504],
'Water_node_30':[0.8595,0.9993],
'Water_node_31':[0.208,0.4504],
'Water_node_32':[0.8595,0.9993],
'Water_node_33':[0.691,0.9993],
'Water_node_34':[0,0.4504],
'Water_node_35':[0,0.2079],
'Water_node_36':[0,0.9993],
'Water_node_37':[0,0.2079],
'Water_node_38':[0.208,0.4504],
'Water_node_39':[0,0.2079],
'Water_node_40':[0,0.2079],
'Water_node_41':[0,0.2079],
'Water_node_42':[0,0.2079],
'Water_node_43':[0.4505,0.9993],
'Water_node_44':[0,0.2079],
'Water_node_45':[0.4505,0.6909],
'Water_node_46':[0,0.2079],
'Water_node_47':[0.4505,0.6909],
'Water_node_48':[0,0.2079]
}
}



In [ ]:
def EV_and_EEVS(alpha_mu_type, b, OMEGA, N, A, T, S, L, R, K, N_star_k, N_k, N_prime_k, A_k, A_prime_k, L_k, v, h, p, M_plus, M_minus, alpha, beta, gamma, g, f, q, c, u, N_k_d, N_k_s, N_k_nod, N_k_d_nod, scarce, rho):

    

    '''

    Expected value

    '''

    print('Expected value')
    model = Model('stochastic_tdINDP')

    '''
    Gurobi parameters

    '''
    utility_type = 'power'

    num_pieces = int(1)
    pieces_length = 1
    num_focus = 3
    # model.setParam('InfUnbdInfo',1)
    model.setParam('OutputFlag', 0)
    model.setParam('FuncPieceRatio', -1)
    model.setParam('FuncPieces',num_pieces)
    model.setParam('FuncPieceLength',pieces_length)
    model.setParam('DualReductions',0)
    model.setParam('NumericFocus',num_focus)
    model.setParam('IntegralityFocus',1) 

    '''
    first stage variables
    '''

    x_star = {(i,j,k,l): model.addVar(vtype = GRB.CONTINUOUS, name = 'x_star_'+str((i,j,k,l))) for k in K for l in L_k[k] for (i,j) in A_k[k] if i in N_k_s[k]}
    w = {(i,k): model.addVar(vtype = GRB.BINARY, name = 'w_'+str((i,k))) for k in K for i in N_k[k]}
    y = {(i,j,k): model.addVar(vtype = GRB.BINARY, name = 'y_'+str((i,j,k))) for k in K for (i,j) in A_k[k]}

    model.update()

    '''
    second stage variables
    '''

    proportional_fairness = model.addVar(vtype = GRB.CONTINUOUS, lb = -GRB.INFINITY, name = 'proportional_fairness')
    phi = {(i,k,l):model.addVar(vtype = GRB.CONTINUOUS, lb=0, ub = abs(b[(i,k,l)]),name="phi_"+str((i,k,l))) for k in K for l in L_k[k] for i in N_k_d[k]}
    log_phi = {(i,k,l):model.addVar(vtype=GRB.CONTINUOUS,lb=-GRB.INFINITY,ub=math.log(abs(b[(i,k,l)])), name="log_phi_"+str((i,k,l))) for k in K for l in L_k[k] for i in N_k_d[k]}
    delta_plus = {(i,k,l): model.addVar(vtype = GRB.CONTINUOUS, name = 'delta_plus_'+str((i,k,l))) for k in K for l in L_k[k] for i in N_k_s[k]}
    delta_minus = {(i,k,l): model.addVar(vtype = GRB.CONTINUOUS, name = 'delta_minus_'+str((i,k,l))) for k in K for l in L_k[k] for i in N_k_d[k]}
    x = {(i,j,k,l): model.addVar(vtype = GRB.CONTINUOUS, name = 'x_'+str((i,j,k,l))) for k in K for l in L_k[k] for (i,j) in A_k[k] if i in N_k_d_nod[k]}

    model.update()

    '''
    Objective functions:

    '''
    if utility_type == 'linear':
        # Linear
        model.addConstr(proportional_fairness == quicksum(log_phi[(i,k,l)] + math.log(0.5) for k in K for l in L_k[k] for i in N_k_d[k]), name = 'proportional_fairness_constraint')
    else:
        # Power
        model.addConstr(proportional_fairness == quicksum(alpha_mu_type[(i,k,l)]*log_phi[(i,k,l)] for k in K for l in L_k[k] for i in N_k_d[k]), name = 'proportional_fairness_constraint')

    model.setObjective(proportional_fairness, GRB.MAXIMIZE)
    model.update()

    '''

    Constraints

    '''


    for k in K:
        for l in L_k[k]:

            for i in N_k_s[k]:
                model.addConstr(quicksum(x_star[(i,j,k,l)] for j in N_k[k] if (i,j) in A_k[k])-(quicksum(x[(j,i,k,l)] for j in N_k_d_nod[k] if (j,i) in A_k[k]) + quicksum(x_star[(j,i,k,l)] for j in N_k_s[k] if (j,i) in A_k[k])) == b[(i,k,l)] - delta_plus[(i,k,l)], name = 'supply_flow_balance_'+str((i,k,l)))


    for k in K:
        for (i,j) in A_k[k]:
            # model.addConstr(y[(i,j,k)] == y[(j,i,k)], name = 'bidir_1_'+str((i,j,k)))
            if i in N_k_s[k]:
                model.addConstr(quicksum(x_star[(i,j,k,l)] for l in L_k[k]) <= u[(i,j,k)]*y[(i,j,k)], name = 'arc_functionality_'+str((i,j,k)))
                model.addConstr(quicksum(x_star[(i,j,k,l)] for l in L_k[k]) <= u[(i,j,k)]*w[(i,k)], name = 'node_i_functionality_'+str((i,j,k)))
                model.addConstr(quicksum(x_star[(i,j,k,l)] for l in L_k[k]) <= u[(i,j,k)]*w[(j,k)], name = 'node_j_functionality_'+str((i,j,k)))

                # model.addConstr(quicksum(x_star[(i,j,k,l)] for l in L_k[k]) <= u[(i,j,k)], name = 'arc_functionality_'+str((i,j,k)))
                # model.addConstr(quicksum(x_star[(i,j,k,l)] for l in L_k[k]) <= u[(i,j,k)], name = 'node_i_functionality_'+str((i,j,k)))
                # model.addConstr(quicksum(x_star[(i,j,k,l)] for l in L_k[k]) <= u[(i,j,k)], name = 'node_j_functionality_'+str((i,j,k)))


    for k in K:
        for k_tilde in K:
            for j in N_k[k_tilde]:
                if (i,j,k,k_tilde) in gamma.keys():
                    model.addConstr(quicksum(w[(i,k)]*gamma[(i,j,k,k_tilde)] for i in N_k[k]) <= w[(j,k_tilde)], name = 'interdependent_functionality_'+str((k,k_tilde,j)))



    for k in K:
        for (i,j) in A_k[k]:
            if i in N_k_d_nod[k]:
                model.addConstr(quicksum(x[(i,j,k,l)] for l in L_k[k]) <= u[(i,j,k)]*y[(i,j,k)], name = 'arc_functionality_'+str((i,j,k)))
                model.addConstr(quicksum(x[(i,j,k,l)] for l in L_k[k]) <= u[(i,j,k)]*w[(i,k)], name = 'node_i_functionality_'+str((i,j,k)))
                model.addConstr(quicksum(x[(i,j,k,l)] for l in L_k[k]) <= u[(i,j,k)]*w[(j,k)], name = 'node_j_functionality_'+str((i,j,k)))
                # model.addConstr(quicksum(x[(i,j,k,l)] for l in L_k[k]) <= u[(i,j,k)], name = 'arc_functionality_'+str((i,j,k)))
                # model.addConstr(quicksum(x[(i,j,k,l)] for l in L_k[k]) <= u[(i,j,k)], name = 'node_i_functionality_'+str((i,j,k)))
                # model.addConstr(quicksum(x[(i,j,k,l)] for l in L_k[k]) <= u[(i,j,k)], name = 'node_j_functionality_'+str((i,j,k)))

    for k in K:
        for l in L_k[k]:
            for i in N_k_d[k]:
                model.addConstr(quicksum(x[(i,j,k,l)] for j in N_k[k] if (i,j) in A_k[k])-(quicksum(x[(j,i,k,l)] for j in N_k_d_nod[k] if (j,i) in A_k[k]) + quicksum(x_star[(j,i,k,l)] for j in N_k_s[k] if (j,i) in A_k[k])) == b[(i,k,l)] + delta_minus[(i,k,l)], name = 'demand_flow_balance_'+str((i,k,l)))
                model.addConstr(phi[(i,k,l)] == abs(b[(i,k,l)]) - delta_minus[(i,k,l)], name = 'capture_phi_'+str((i,k,l)))
                model.addGenConstrLog(phi[(i,k,l)], log_phi[(i,k,l)], name = 'log_phi_constr_'+str((i,k,l)))

            for i in N_k_nod[k]:
                model.addConstr(quicksum(x[(i,j,k,l)] for j in N_k[k] if (i,j) in A_k[k])-(quicksum(x[(j,i,k,l)] for j in N_k_d_nod[k] if (j,i) in A_k[k]) + quicksum(x_star[(j,i,k,l)] for j in N_k_s[k] if (j,i) in A_k[k])) == 0, name = 'transshipment_flow_balance_'+str((i,k,l)))

    model.update()

    model.optimize()

    x_star_EV = {var.varName:var.x for var in model.getVars()}
    # OF_EV = model.objVal # to be computed with the natural log
    OF_EV = sum(alpha_mu_type[(i,k,l)]*math.log(x_star_EV["phi_"+str((i,k,l))]) for k in K for l in L_k[k] for i in N_k_d[k] if x_star_EV["phi_"+str((i,k,l))] > 0)

    del model

    ''' 

    Expectation of the expected value

    '''

    x_star_EEVS = {}
    OF_omega = []
    for omega in OMEGA:
        print(f'Expectation of the expected value, scenario {omega+1} out of {len(OMEGA)}')

        model = Model('stochastic_tdINDP')

        '''
        Gurobi parameters

        '''
        # model.setParam('InfUnbdInfo',1)
        model.setParam('OutputFlag', 0)
        model.setParam('FuncPieceRatio', -1)
        model.setParam('FuncPieces',num_pieces)
        model.setParam('FuncPieceLength',pieces_length)
        model.setParam('DualReductions',0)
        model.setParam('NumericFocus',num_focus)
        # model.setParam(GRB.Param.NonConvex, 2)
        # model.setParam('MIPGap',0.01)
        # model.setParam('OptimalityTol',0.01)
        model.setParam('IntegralityFocus',1) 

        '''
        first stage variables
        '''

        x_star = {(i,j,k,l): model.addVar(vtype = GRB.CONTINUOUS, name = 'x_star_'+str((i,j,k,l))) for k in K for l in L_k[k] for (i,j) in A_k[k] if i in N_k_s[k]}
        w = {(i,k): model.addVar(vtype = GRB.BINARY, name = 'w_'+str((i,k))) for k in K for i in N_k[k]}
        y = {(i,j,k): model.addVar(vtype = GRB.BINARY, name = 'y_'+str((i,j,k))) for k in K for (i,j) in A_k[k]}

        model.update()

        '''
        second stage variables
        '''

        proportional_fairness = model.addVar(vtype = GRB.CONTINUOUS, lb = -GRB.INFINITY, name = 'proportional_fairness_'+str(omega))
        phi = {(i,k,l,omega):model.addVar(vtype = GRB.CONTINUOUS, lb=0, ub = abs(b[(i,k,l)]),name="phi_"+str((i,k,l,omega))) for k in K for l in L_k[k] for i in N_k_d[k]}
        log_phi = {(i,k,l,omega):model.addVar(vtype=GRB.CONTINUOUS,lb=-GRB.INFINITY,ub=math.log(abs(b[(i,k,l)])), name="log_phi_"+str((i,k,l,omega))) for k in K for l in L_k[k] for i in N_k_d[k]}
        delta_plus = {(i,k,l,omega): model.addVar(vtype = GRB.CONTINUOUS, name = 'delta_plus_'+str((i,k,l,omega))) for k in K for l in L_k[k] for i in N_k_s[k]}
        delta_minus = {(i,k,l,omega): model.addVar(vtype = GRB.CONTINUOUS, name = 'delta_minus_'+str((i,k,l,omega))) for k in K for l in L_k[k] for i in N_k_d[k]}
        x = {(i,j,k,l,omega): model.addVar(vtype = GRB.CONTINUOUS, name = 'x_'+str((i,j,k,l,omega))) for k in K for l in L_k[k] for (i,j) in A_k[k] if i in N_k_d_nod[k]}

        model.update()

        '''
        Objective functions:

        '''
        if utility_type == 'linear':
            # Linear
            model.addConstr(proportional_fairness == quicksum(log_phi[(i,k,l,omega)] + math.log(alpha[(i,k,l,omega)]) for k in K for l in L_k[k] for i in N_k_d[k]), name = 'proportional_fairness_constraint')
        else:
            # Power
            model.addConstr(proportional_fairness == quicksum(alpha[(i,k,l,omega)]*log_phi[(i,k,l,omega)] for k in K for l in L_k[k] for i in N_k_d[k]), name = 'proportional_fairness_constraint')

        model.setObjective(proportional_fairness, GRB.MAXIMIZE)
        model.update()

        '''

        Constraints

        '''

        for k in K:
            for l in L_k[k]:

                for i in N_k_s[k]:
                    model.addConstr(quicksum(x_star[(i,j,k,l)] for j in N_k[k] if (i,j) in A_k[k])-(quicksum(x[(j,i,k,l,omega)] for j in N_k_d_nod[k] if (j,i) in A_k[k]) + quicksum(x_star[(j,i,k,l)] for j in N_k_s[k] if (j,i) in A_k[k])) == b[(i,k,l)] - delta_plus[(i,k,l,omega)], name = 'supply_flow_balance_'+str((i,k,l,omega)))


        for k in K:
            for (i,j) in A_k[k]:
                # model.addConstr(y[(i,j,k)] == y[(j,i,k)], name = 'bidir_1_'+str((i,j,k)))
                if i in N_k_s[k]:
                    model.addConstr(quicksum(x_star[(i,j,k,l)] for l in L_k[k]) <= u[(i,j,k)]*y[(i,j,k)], name = 'arc_functionality_'+str((i,j,k)))
                    model.addConstr(quicksum(x_star[(i,j,k,l)] for l in L_k[k]) <= u[(i,j,k)]*w[(i,k)], name = 'node_i_functionality_'+str((i,j,k)))
                    model.addConstr(quicksum(x_star[(i,j,k,l)] for l in L_k[k]) <= u[(i,j,k)]*w[(j,k)], name = 'node_j_functionality_'+str((i,j,k)))

                    # model.addConstr(quicksum(x_star[(i,j,k,l)] for l in L_k[k]) <= u[(i,j,k)], name = 'arc_functionality_'+str((i,j,k)))
                    # model.addConstr(quicksum(x_star[(i,j,k,l)] for l in L_k[k]) <= u[(i,j,k)], name = 'node_i_functionality_'+str((i,j,k)))
                    # model.addConstr(quicksum(x_star[(i,j,k,l)] for l in L_k[k]) <= u[(i,j,k)], name = 'node_j_functionality_'+str((i,j,k)))


        for k in K:
            for k_tilde in K:
                for j in N_k[k_tilde]:
                    if (i,j,k,k_tilde) in gamma.keys():
                        model.addConstr(quicksum(w[(i,k)]*gamma[(i,j,k,k_tilde)] for i in N_k[k]) <= w[(j,k_tilde)], name = 'interdependent_functionality_'+str((k,k_tilde,j)))



        for k in K:
            for (i,j) in A_k[k]:
                if i in N_k_d_nod[k]:
                    model.addConstr(quicksum(x[(i,j,k,l,omega)] for l in L_k[k]) <= u[(i,j,k)]*y[(i,j,k)], name = 'arc_functionality_'+str((i,j,k,omega)))
                    model.addConstr(quicksum(x[(i,j,k,l,omega)] for l in L_k[k]) <= u[(i,j,k)]*w[(i,k)], name = 'node_i_functionality_'+str((i,j,k,omega)))
                    model.addConstr(quicksum(x[(i,j,k,l,omega)] for l in L_k[k]) <= u[(i,j,k)]*w[(j,k)], name = 'node_j_functionality_'+str((i,j,k,omega)))
                    # model.addConstr(quicksum(x[(i,j,k,l,omega)] for l in L_k[k]) <= u[(i,j,k)], name = 'arc_functionality_'+str((i,j,k)))
                    # model.addConstr(quicksum(x[(i,j,k,l,omega)] for l in L_k[k]) <= u[(i,j,k)], name = 'node_i_functionality_'+str((i,j,k)))
                    # model.addConstr(quicksum(x[(i,j,k,l,omega)] for l in L_k[k]) <= u[(i,j,k)], name = 'node_j_functionality_'+str((i,j,k)))

        for k in K:
            for l in L_k[k]:
                for i in N_k_d[k]:
                    model.addConstr(quicksum(x[(i,j,k,l,omega)] for j in N_k[k] if (i,j) in A_k[k])-(quicksum(x[(j,i,k,l,omega)] for j in N_k_d_nod[k] if (j,i) in A_k[k]) + quicksum(x_star[(j,i,k,l)] for j in N_k_s[k] if (j,i) in A_k[k])) == b[(i,k,l)] + delta_minus[(i,k,l,omega)], name = 'demand_flow_balance_'+str((i,k,l,omega)))
                    model.addConstr(phi[(i,k,l,omega)] == abs(b[(i,k,l)]) - delta_minus[(i,k,l,omega)], name = 'capture_phi_'+str((i,k,l,omega)))
                    model.addGenConstrLog(phi[(i,k,l,omega)], log_phi[(i,k,l,omega)], name = 'log_phi_constr_'+str((i,k,l,omega)))

                for i in N_k_nod[k]:
                    model.addConstr(quicksum(x[(i,j,k,l,omega)] for j in N_k[k] if (i,j) in A_k[k])-(quicksum(x[(j,i,k,l,omega)] for j in N_k_d_nod[k] if (j,i) in A_k[k]) + quicksum(x_star[(j,i,k,l)] for j in N_k_s[k] if (j,i) in A_k[k])) == 0, name = 'transshipment_flow_balance_'+str((i,k,l,omega)))

        for k in K:
            for (i,j) in A_k[k]:
                for l in L_k[k]:
                    if i in N_k_s[k]:
                        model.addConstr(model.getVarByName('x_star_'+str((i,j,k,l))) == x_star_EV['x_star_'+str((i,j,k,l))], name = 'fix_first_stage_x_star_'+str((i,j,k,l)))

        model.update()

        model.optimize()

        x_star_EEVS[omega] = {var.varName:var.x for var in model.getVars()}
        OF_omega_obj_val = sum(alpha[(i,k,l,omega)]*math.log(x_star_EEVS[omega]["phi_"+str((i,k,l,omega))]) for k in K for l in L_k[k] for i in N_k_d[k] if x_star_EEVS[omega]["phi_"+str((i,k,l,omega))] > 0)
        OF_omega.append(OF_omega_obj_val) 

        del model

    OF_EEVS = sum(OF for OF in OF_omega)/len(OMEGA)

    return x_star_EV, OF_EV, x_star_EEVS, OF_EEVS

# Here could go a function that solves computes the EEVS by plugging the correspondent allocation (phi) variables in the weighted (per scenario) objective function after solving EV $$\downarrow$$

In [26]:
#def fixOnlyFirstStageEEVS()... 
# sum(alpha[(i,k,l,omega)]*math.log(x_star_EV["phi_"+str((i,k,l,omega))] for k in K for l in L_k[k] for i in N_k_d[k])

In [27]:
def WS(b, OMEGA, N, A, T, S, L, R, K, N_star_k, N_k, N_prime_k, A_k, A_prime_k, L_k, v, h, p, M_plus, M_minus, alpha, beta, gamma, g, f, q, c, u, N_k_d, N_k_s, N_k_nod, N_k_d_nod, scarce, rho, x_start = {}):

    x_star_WS = {}
    OF_omega = []
    for omega in OMEGA:
        print(f'Wait and see, scenario {omega+1} out of {len(OMEGA)}')

        model = Model('stochastic_tdINDP')

        '''
        Gurobi parameters

        '''
        utility_type = 'power'

        num_pieces = int(1)
        pieces_length = 1
        num_focus = 3
        # model.setParam('InfUnbdInfo',1)
        model.setParam('OutputFlag', 0)
        model.setParam('FuncPieceRatio', -1)
        model.setParam('FuncPieces',num_pieces)
        model.setParam('FuncPieceLength',pieces_length)
        model.setParam('DualReductions',0)
        model.setParam('NumericFocus',num_focus)
        # model.setParam(GRB.Param.NonConvex, 2)
        # model.setParam('MIPGap',0.01)
        # model.setParam('OptimalityTol',0.01)
        model.setParam('IntegralityFocus',1) 

        '''
        first stage variables
        '''

        x_star = {(i,j,k,l): model.addVar(vtype = GRB.CONTINUOUS, name = 'x_star_'+str((i,j,k,l))) for k in K for l in L_k[k] for (i,j) in A_k[k] if i in N_k_s[k]}
        w = {(i,k): model.addVar(vtype = GRB.BINARY, name = 'w_'+str((i,k))) for k in K for i in N_k[k]}
        y = {(i,j,k): model.addVar(vtype = GRB.BINARY, name = 'y_'+str((i,j,k))) for k in K for (i,j) in A_k[k]}

        model.update()

        '''
        second stage variables
        '''

        proportional_fairness = model.addVar(vtype = GRB.CONTINUOUS, lb = -GRB.INFINITY, name = 'proportional_fairness_'+str(omega))
        phi = {(i,k,l,omega):model.addVar(vtype = GRB.CONTINUOUS, lb=0, ub = abs(b[(i,k,l)]),name="phi_"+str((i,k,l,omega))) for k in K for l in L_k[k] for i in N_k_d[k]}
        log_phi = {(i,k,l,omega):model.addVar(vtype=GRB.CONTINUOUS,lb=-GRB.INFINITY,ub=math.log(abs(b[(i,k,l)])), name="log_phi_"+str((i,k,l,omega))) for k in K for l in L_k[k] for i in N_k_d[k]}
        delta_plus = {(i,k,l,omega): model.addVar(vtype = GRB.CONTINUOUS, name = 'delta_plus_'+str((i,k,l,omega))) for k in K for l in L_k[k] for i in N_k_s[k]}
        delta_minus = {(i,k,l,omega): model.addVar(vtype = GRB.CONTINUOUS, name = 'delta_minus_'+str((i,k,l,omega))) for k in K for l in L_k[k] for i in N_k_d[k]}
        x = {(i,j,k,l,omega): model.addVar(vtype = GRB.CONTINUOUS, name = 'x_'+str((i,j,k,l,omega))) for k in K for l in L_k[k] for (i,j) in A_k[k] if i in N_k_d_nod[k]}

        model.update()

        '''
        Objective functions:

        '''
        if utility_type == 'linear':
            # Linear
            model.addConstr(proportional_fairness == quicksum(log_phi[(i,k,l,omega)] + math.log(alpha[(i,k,l,omega)]) for k in K for l in L_k[k] for i in N_k_d[k]), name = 'proportional_fairness_constraint')
        else:
            # Power
            model.addConstr(proportional_fairness == quicksum(alpha[(i,k,l,omega)]*log_phi[(i,k,l,omega)] for k in K for l in L_k[k] for i in N_k_d[k]), name = 'proportional_fairness_constraint')

        model.setObjective(proportional_fairness, GRB.MAXIMIZE)
        model.update()

        '''

        Constraints

        '''

        for k in K:
            for l in L_k[k]:
                for i in N_k_s[k]:
                    model.addConstr(quicksum(x_star[(i,j,k,l)] for j in N_k[k] if (i,j) in A_k[k])-(quicksum(x[(j,i,k,l,omega)] for j in N_k_d_nod[k] if (j,i) in A_k[k]) + quicksum(x_star[(j,i,k,l)] for j in N_k_s[k] if (j,i) in A_k[k])) == b[(i,k,l)] - delta_plus[(i,k,l,omega)], name = 'supply_flow_balance_'+str((i,k,l,omega)))


        for k in K:
            for (i,j) in A_k[k]:
                # model.addConstr(y[(i,j,k)] == y[(j,i,k)], name = 'bidir_1_'+str((i,j,k)))
                if i in N_k_s[k]:
                    model.addConstr(quicksum(x_star[(i,j,k,l)] for l in L_k[k]) <= u[(i,j,k)]*y[(i,j,k)], name = 'arc_functionality_'+str((i,j,k)))
                    model.addConstr(quicksum(x_star[(i,j,k,l)] for l in L_k[k]) <= u[(i,j,k)]*w[(i,k)], name = 'node_i_functionality_'+str((i,j,k)))
                    model.addConstr(quicksum(x_star[(i,j,k,l)] for l in L_k[k]) <= u[(i,j,k)]*w[(j,k)], name = 'node_j_functionality_'+str((i,j,k)))

                    # model.addConstr(quicksum(x_star[(i,j,k,l)] for l in L_k[k]) <= u[(i,j,k)], name = 'arc_functionality_'+str((i,j,k)))
                    # model.addConstr(quicksum(x_star[(i,j,k,l)] for l in L_k[k]) <= u[(i,j,k)], name = 'node_i_functionality_'+str((i,j,k)))
                    # model.addConstr(quicksum(x_star[(i,j,k,l)] for l in L_k[k]) <= u[(i,j,k)], name = 'node_j_functionality_'+str((i,j,k)))


        for k in K:
            for k_tilde in K:
                for j in N_k[k_tilde]:
                    if (i,j,k,k_tilde) in gamma.keys():
                        model.addConstr(quicksum(w[(i,k)]*gamma[(i,j,k,k_tilde)] for i in N_k[k]) <= w[(j,k_tilde)], name = 'interdependent_functionality_'+str((k,k_tilde,j)))



        for k in K:
            for (i,j) in A_k[k]:
                if i in N_k_d_nod[k]:
                    model.addConstr(quicksum(x[(i,j,k,l,omega)] for l in L_k[k]) <= u[(i,j,k)]*y[(i,j,k)], name = 'arc_functionality_'+str((i,j,k,omega)))
                    model.addConstr(quicksum(x[(i,j,k,l,omega)] for l in L_k[k]) <= u[(i,j,k)]*w[(i,k)], name = 'node_i_functionality_'+str((i,j,k,omega)))
                    model.addConstr(quicksum(x[(i,j,k,l,omega)] for l in L_k[k]) <= u[(i,j,k)]*w[(j,k)], name = 'node_j_functionality_'+str((i,j,k,omega)))
                    # model.addConstr(quicksum(x[(i,j,k,l,omega)] for l in L_k[k]) <= u[(i,j,k)], name = 'arc_functionality_'+str((i,j,k)))
                    # model.addConstr(quicksum(x[(i,j,k,l,omega)] for l in L_k[k]) <= u[(i,j,k)], name = 'node_i_functionality_'+str((i,j,k)))
                    # model.addConstr(quicksum(x[(i,j,k,l,omega)] for l in L_k[k]) <= u[(i,j,k)], name = 'node_j_functionality_'+str((i,j,k)))

        for k in K:
            for l in L_k[k]:
                for i in N_k_d[k]:
                    model.addConstr(quicksum(x[(i,j,k,l,omega)] for j in N_k[k] if (i,j) in A_k[k])-(quicksum(x[(j,i,k,l,omega)] for j in N_k_d_nod[k] if (j,i) in A_k[k]) + quicksum(x_star[(j,i,k,l)] for j in N_k_s[k] if (j,i) in A_k[k])) == b[(i,k,l)] + delta_minus[(i,k,l,omega)], name = 'demand_flow_balance_'+str((i,k,l,omega)))
                    model.addConstr(phi[(i,k,l,omega)] == abs(b[(i,k,l)]) - delta_minus[(i,k,l,omega)], name = 'capture_phi_'+str((i,k,l,omega)))
                    model.addGenConstrLog(phi[(i,k,l,omega)], log_phi[(i,k,l,omega)], name = 'log_phi_constr_'+str((i,k,l,omega)))

                for i in N_k_nod[k]:
                    model.addConstr(quicksum(x[(i,j,k,l,omega)] for j in N_k[k] if (i,j) in A_k[k])-(quicksum(x[(j,i,k,l,omega)] for j in N_k_d_nod[k] if (j,i) in A_k[k]) + quicksum(x_star[(j,i,k,l)] for j in N_k_s[k] if (j,i) in A_k[k])) == 0, name = 'transshipment_flow_balance_'+str((i,k,l,omega)))

        if x_start:
            for k in K:
                for l in L_k[k]:
                    for i in N_k_d[k]:
                        model.getVarByName('phi_'+str((i,k,l,omega))).Start = x_star_EV['phi_'+str((i,k,l))]
                        model.update()

        model.update()

        model.optimize()

        x_star_WS[omega] = {var.varName:var.x for var in model.getVars()}
        OF_omega_obj_val = sum(alpha[(i,k,l,omega)]*math.log(x_star_WS[omega]["phi_"+str((i,k,l,omega))]) for k in K for l in L_k[k] for i in N_k_d[k] if x_star_WS[omega]["phi_"+str((i,k,l,omega))] > 0)
        OF_omega.append(OF_omega_obj_val) 

        del model

    OF_WS = sum(OF for OF in OF_omega)/len(OMEGA)

    return x_star_WS, OF_WS

In [33]:
def DE(b, OMEGA, N, A, T, S, L, R, K, N_star_k, N_k, N_prime_k, A_k, A_prime_k, L_k, v, h, p, M_plus, M_minus, alpha, beta, gamma, g, f, q, c, u, N_k_d, N_k_s, N_k_nod, N_k_d_nod, scarce, rho, x_start = {}):

    print(f'Deterministic equivalent')

    model = Model('stochastic_tdINDP')


    '''
    Gurobi parameters

    '''

    utility_type = 'power'

    num_pieces = int(1)
    pieces_length = 1
    num_focus = 3

    # model.setParam('InfUnbdInfo',1)
    # model.setParam('OutputFlag', 0)
    model.setParam('FuncPieceRatio', -1)
    model.setParam('FuncPieces',num_pieces)
    model.setParam('FuncPieceLength',pieces_length)
    model.setParam('DualReductions',0)
    model.setParam('NumericFocus',num_focus)
    model.setParam('IntegralityFocus',1) 

    '''
    first stage variables
    '''

    x_star = {(i,j,k,l): model.addVar(vtype = GRB.CONTINUOUS, name = 'x_star_'+str((i,j,k,l))) for k in K for l in L_k[k] for (i,j) in A_k[k] if i in N_k_s[k]}
    w = {(i,k): model.addVar(vtype = GRB.BINARY, name = 'w_'+str((i,k))) for k in K for i in N_k[k]}
    y = {(i,j,k): model.addVar(vtype = GRB.BINARY, name = 'y_'+str((i,j,k))) for k in K for (i,j) in A_k[k]}

    model.update()

    '''
    second stage variables
    '''

    proportional_fairness = {omega:model.addVar(vtype = GRB.CONTINUOUS, lb = -GRB.INFINITY, name = 'proportional_fairness_'+str(omega)) for omega in OMEGA}
    phi = {(i,k,l,omega):model.addVar(vtype = GRB.CONTINUOUS, lb=0, ub = abs(b[(i,k,l)]),name="phi_"+str((i,k,l,omega))) for k in K for l in L_k[k] for i in N_k_d[k] for omega in OMEGA}
    log_phi = {(i,k,l,omega):model.addVar(vtype=GRB.CONTINUOUS,lb=-GRB.INFINITY,ub=math.log(abs(b[(i,k,l)])), name="log_phi_"+str((i,k,l,omega))) for k in K for l in L_k[k] for i in N_k_d[k] for omega in OMEGA}
    delta_plus = {(i,k,l,omega): model.addVar(vtype = GRB.CONTINUOUS, name = 'delta_plus_'+str((i,k,l,omega))) for k in K for l in L_k[k] for i in N_k_s[k] for omega in OMEGA}
    delta_minus = {(i,k,l,omega): model.addVar(vtype = GRB.CONTINUOUS, name = 'delta_minus_'+str((i,k,l,omega))) for k in K for l in L_k[k] for i in N_k_d[k] for omega in OMEGA}
    x = {(i,j,k,l,omega): model.addVar(vtype = GRB.CONTINUOUS, name = 'x_'+str((i,j,k,l,omega))) for k in K for l in L_k[k] for (i,j) in A_k[k] if i in N_k_d_nod[k] for omega in OMEGA}

    model.update()

    '''
    Objective functions:

    '''
    if utility_type == 'linear':
        # Linear
        for omega in OMEGA:
            model.addConstr(proportional_fairness[omega] == quicksum(log_phi[(i,k,l,omega)] + math.log(alpha[(i,k,l,omega)]) for k in K for l in L_k[k] for i in N_k_d[k]), name = 'proportional_fairness_constraint')
    else:
        # Power
        for omega in OMEGA:
            model.addConstr(proportional_fairness[omega] == quicksum(alpha[(i,k,l,omega)]*log_phi[(i,k,l,omega)] for k in K for l in L_k[k] for i in N_k_d[k]), name = 'proportional_fairness_constraint')

    model.setObjective(quicksum(rho[omega]*proportional_fairness[omega] for omega in OMEGA), GRB.MAXIMIZE)
    model.update()

    '''

    Constraints

    '''

    for k in K:
        for l in L_k[k]:

            for i in N_k_s[k]:
                for omega in OMEGA:
                    model.addConstr(quicksum(x_star[(i,j,k,l)] for j in N_k[k] if (i,j) in A_k[k])-(quicksum(x[(j,i,k,l,omega)] for j in N_k_d_nod[k] if (j,i) in A_k[k]) + quicksum(x_star[(j,i,k,l)] for j in N_k_s[k] if (j,i) in A_k[k])) == b[(i,k,l)] - delta_plus[(i,k,l,omega)], name = 'supply_flow_balance_'+str((i,k,l,omega)))


    for k in K:
        for (i,j) in A_k[k]:
            if i in N_k_s[k]:
                model.addConstr(quicksum(x_star[(i,j,k,l)] for l in L_k[k]) <= u[(i,j,k)]*y[(i,j,k)], name = 'arc_functionality_'+str((i,j,k)))
                model.addConstr(quicksum(x_star[(i,j,k,l)] for l in L_k[k]) <= u[(i,j,k)]*w[(i,k)], name = 'node_i_functionality_'+str((i,j,k)))
                model.addConstr(quicksum(x_star[(i,j,k,l)] for l in L_k[k]) <= u[(i,j,k)]*w[(j,k)], name = 'node_j_functionality_'+str((i,j,k)))


    for k in K:
        for k_tilde in K:
            for j in N_k[k_tilde]:
                if (i,j,k,k_tilde) in gamma.keys():
                    model.addConstr(quicksum(w[(i,k)]*gamma[(i,j,k,k_tilde)] for i in N_k[k]) <= w[(j,k_tilde)], name = 'interdependent_functionality_'+str((k,k_tilde,j)))



    for k in K:
        for (i,j) in A_k[k]:
            for omega in OMEGA:
                if i in N_k_d_nod[k]:
                    model.addConstr(quicksum(x[(i,j,k,l,omega)] for l in L_k[k]) <= u[(i,j,k)]*y[(i,j,k)], name = 'arc_functionality_'+str((i,j,k,omega)))
                    model.addConstr(quicksum(x[(i,j,k,l,omega)] for l in L_k[k]) <= u[(i,j,k)]*w[(i,k)], name = 'node_i_functionality_'+str((i,j,k,omega)))
                    model.addConstr(quicksum(x[(i,j,k,l,omega)] for l in L_k[k]) <= u[(i,j,k)]*w[(j,k)], name = 'node_j_functionality_'+str((i,j,k,omega)))

    for k in K:
        for l in L_k[k]:
            for i in N_k_d[k]:
                for omega in OMEGA:
                    model.addConstr(quicksum(x[(i,j,k,l,omega)] for j in N_k[k] if (i,j) in A_k[k])-(quicksum(x[(j,i,k,l,omega)] for j in N_k_d_nod[k] if (j,i) in A_k[k]) + quicksum(x_star[(j,i,k,l)] for j in N_k_s[k] if (j,i) in A_k[k])) == b[(i,k,l)] + delta_minus[(i,k,l,omega)], name = 'demand_flow_balance_'+str((i,k,l,omega)))
                    model.addConstr(phi[(i,k,l,omega)] == abs(b[(i,k,l)]) - delta_minus[(i,k,l,omega)], name = 'capture_phi_'+str((i,k,l,omega)))
                    model.addGenConstrLog(phi[(i,k,l,omega)], log_phi[(i,k,l,omega)], name = 'log_phi_constr_'+str((i,k,l,omega)))

            for i in N_k_nod[k]:
                for omega in OMEGA:
                    model.addConstr(quicksum(x[(i,j,k,l,omega)] for j in N_k[k] if (i,j) in A_k[k])-(quicksum(x[(j,i,k,l,omega)] for j in N_k_d_nod[k] if (j,i) in A_k[k]) + quicksum(x_star[(j,i,k,l)] for j in N_k_s[k] if (j,i) in A_k[k])) == 0, name = 'transshipment_flow_balance_'+str((i,k,l,omega)))

    # if x_start:
    #     # model.addConstr(proportional_fairness <= x_start['proportional_fairness'])
    #     for k in K:
    #         for l in L_k[k]:
    #             for i in N_k_d[k]:
    #                 for omega in OMEGA:
    #                     model.getVarByName('phi_'+str((i,k,l,omega))).Start = x_star_EV['phi_'+str((i,k,l))]
    #                     model.update()
    model.update()

    model.params.LogFile=f'LOG_FILE_scarce_{scarce}_utility_type_{utility_type}_num_scenarios_{len(OMEGA)}_equal_probabilities_{True}_nat_log_linear_pieces_length_{pieces_length}'
    model.optimize()

    # print(f'\nscarce: {scarce*100}, Utility type: {utility_type}, Num. scenarios: {len(OMEGA)}, Equal probabilities?: {same_probs}, nat. log. linear pieces length: {pieces_length}')
    
    proportional_fairness_values = [sum(math.log(model.getVarByName('phi_'+str(i,k,l,omega)).x) for k in K for l in L_k[k] for i in N_k_d[k] if model.getVarByName('phi_'+str(i,k,l,omega)).x > 0) for omega in OMEGA]

    # Calculate mean and variance
    mean_value = np.mean(proportional_fairness_values)
    variance_value = np.var(proportional_fairness_values)
    std_value = np.std(proportional_fairness_values)
    
    for k in K:
        # print(f'\n---------------Network: {k}-----------------')
        for l in L_k[k]:
            for i in N_k_d[k]:
                # print()
                # print(f'node: {i}\n')
                allocation = []
                for omega in OMEGA:
                    # print(f'allocation value in scenario {omega}: {model.getVarByName("phi_"+str((i,k,l,omega))).x}')
                    allocation.append(model.getVarByName("phi_"+str((i,k,l,omega))).x)
                # Calculate mean and variance
                mean_value = np.mean(allocation)
                variance_value = np.var(allocation)
                std_value = np.std(allocation)
                # print()
                # print(f'mean allocation value: {mean_value}, allocation variance: {variance_value}, allocation std: {std_value}')


    results = {}

    results['proportional_fairness'] = [sum(math.log(model.getVarByName('phi_'+str((i,k,l,omega))).x) for k in K for l in L_k[k] for i in N_k_d[k] if model.getVarByName('phi_'+str(i,k,l,omega)).x > 0) for omega in OMEGA]

    for k in K:
        for l in L_k[k]:
            for i in N_k_d[k]:
                results['alpha_'+str((i,k,l))] = [alpha[(i,k,l,omega)] for omega in OMEGA]
                results['phi_'+str((i,k,l))] = [model.getVarByName('phi_'+str((i,k,l,omega))).x for omega in OMEGA]
                results['utility_of_phi_'+str((i,k,l))] = [max((model.getVarByName('phi_'+str((i,k,l,omega))).x**alpha[(i,k,l,omega)]).real,0) for omega in OMEGA]
                results['utility_of_demand_'+str((i,k,l))] = [abs(b[(i,k,l)])**alpha[(i,k,l,omega)] for omega in OMEGA]

    for k in K:
        for l in L_k[k]:
            for i in N_k_d[k]:
                results['log_phi_'+str((i,k,l))] = [math.log(model.getVarByName('phi_'+str((i,k,l,omega))).x) for omega in OMEGA]

    for k in K:
        for l in L_k[k]:
            for i in N_k_s[k]:
                results['delta_plus_'+str((i,k,l))] = [model.getVarByName('delta_plus_'+str((i,k,l,omega))).x for omega in OMEGA]

    for k in K:
        for l in L_k[k]:
            for i in N_k_d[k]:
                results['delta_minus_'+str((i,k,l))] = [model.getVarByName('delta_minus_'+str((i,k,l,omega))).x for omega in OMEGA]

    for k in K:
        for l in L_k[k]:
            for i in A_k[k]:
                if i in N_k_d_nod[k]:
                    results['x_'+str((i,j,k,l))] = [model.getVarByName('x_'+str((i,j,k,l,omega))).x for omega in OMEGA]			

    for k in K:
        for l in L_k[k]:
            for i in A_k[k]:
                if i in N_k_s[k]:
                    results['x_star_'+str((i,j,k,l))] = [model.getVarByName('x_star_'+str((i,j,k,l))).x for omega in OMEGA]

    # for k in K:
    # 	for l in L_k[k]:
    # 		for i in N_k[k]:
    # 			results['w_'+str((i,k))] = [model.getVarByName('w_'+str((i,k))).x for omega in OMEGA]

    # for k in K:
    # 	for (i,j) in A_k[k]:
    # 		results['y_'+str((i,j,k))] = [model.getVarByName('y_'+str((i,j,k))).x for omega in OMEGA]

    results = pd.DataFrame(results, columns = results.keys())

    results = results.transpose()

    def compute_statistics(row):
        return pd.Series({
            'mean': np.mean(row),
            'std': np.std(row),
            'min': np.min(row),
            'max': np.max(row),
            'median': np.median(row),
            'mode': row.mode().iloc[0],
            'q1': np.percentile(row, 25),
            'q2': np.percentile(row, 50),
            'q3': np.percentile(row, 75)
        })

    # Apply the function to each row
    summary_stats = results.apply(compute_statistics, axis=1)

    # Concatenate the original DataFrame with the new statistics
    results = pd.concat([results, summary_stats], axis=1)

    results.to_csv(f'scarce_{scarce}_utility_type_{utility_type}_num_scenarios_{len(OMEGA)}_equal_probabilities_{True}_nat_log_linear_pieces_length_{pieces_length}.csv')

    if len(OMEGA) == 1:
        x_start = {(i,k,l):model.getVarByName('phi_'+str((i,k,l,omega))).x for k in K for l in L_k[k] for i in N_k_d[k]}
        with open(f"utiltiy_{utility_type}_scarce_{scarce}_x_start.txt", "w") as file:
            file.write(f'{x_start}')

    x_star_DE = {var.varName:var.x for var in model.getVars()}
    OF_DE = sum(rho[omega]*alpha[(i,k,l,omega)]*math.log(x_star_DE["phi_"+str((i,k,l,omega))]) for k in K for l in L_k[k] for i in N_k_d[k] for omega in OMEGA if x_star_DE["phi_"+str((i,k,l,omega))] > 0)
    return x_star_DE, OF_DE

In [ ]:
for scarce in [0.25, 0.5, 0.75, 1]:

    print(f'\nscarce: {scarce}\n')
    
    utility_type = 'power'

    num_pieces = int(1)
    pieces_length = 1
    num_focus = 3
    
    N, A, T, S, L, R, K, N_star_k, N_k, N_prime_k, A_k, A_prime_k, L_k, v, h, p, M_plus, M_minus, alpha, beta, gamma, g, f, q, c, u, b = tdINDP_parameters_build.load_tdINDP_parameters()

    T = [0]

    u = tdINDP_parameters_build.find_capacities()

    N_k_d = {k:[i for i in N_k[k] if b[(i,k,k,0)] < 0] for k in K}
    N_k_s = {k:[i for i in N_k[k] if b[(i,k,k,0)] > 0]  for k in K}
    N_k_nod = {k:[i for i in N_k[k] if b[(i,k,k,0)] == 0] for k in K}
    N_k_d_nod = {k:list(set(N_k_d[k]+N_k_nod[k])) for k in K}  
    
    N_prime_k, A_prime_k = tdINDP_parameters_build.disrput(0)

    b_temp = {}
    for (i,k,l,_) in b.keys():
        b_temp[(i,k,l)] = b[(i,k,l,0)]
    b = b_temp
    u_temp = {}
    for (i,j,k,_) in u.keys():
        u_temp[(i,j,k)] = u[(i,j,k,0)]
    u = u_temp

    for k in K:
        thy_sum = 0
        for i in N_k_s[k]:
            b[(i,k,k)] = b[(i,k,k)]*scarce

    demand = {k:{i:abs(b[(i,k,l)]) for l in L_k[k] for i in N_k_d[k]} for k in K} 

    random.seed(42)

    same_probs = True
    # same_probs = False

    num_scenarios = 150
    OMEGA = [omega for omega in range(num_scenarios)]

    rho = {omega:1/len(OMEGA) for omega in OMEGA}
    
    rho_sum = sum(rho[omega] for omega in OMEGA)

    if len(OMEGA) > 1:
        if SVI:
            alpha = {(i,k,l,omega): random.uniform(SVI[k][i][0], SVI[k][i][1]) for k in K for l in L_k[k] for i in N_k_d[k] for omega in OMEGA}
        else:
            alpha = {(i,k,l,omega): random.uniform(0, 1) for k in K for l in L_k[k] for i in N_k_d[k] for omega in OMEGA}
    else:
        alpha = {(i,k,l,omega): 1 for k in K for l in L_k[k] for i in N_k_d[k] for omega in OMEGA}
    print()

    alpha_mu = {(i,k,l): (SVI[k][i][0] + SVI[k][i][1])/2 for k in K for l in L_k[k] for i in N_k_d[k]}
    # print(f"alpha_mu: \n {alpha_mu}")
    alpha_mu_hat = {(i,k,l): sum(alpha[(i,k,l,omega)]*rho[omega] for omega in OMEGA) for k in K for l in L_k[k] for i in N_k_d[k]}
    # print(f"alpha_mu_hat: \n {alpha_mu_hat}")
    
    print('Expected value population\n')        
    x_star_EV_alpha_mu, OF_EV_alpha_mu, x_star_EEVS_alpha_mu, OF_EEVS_alpha_mu = EV_and_EEVS(alpha_mu_type=alpha_mu, b = b, OMEGA = OMEGA, N=N, A=A, T=T, S=S, L=L, R=R, K=K, N_star_k=N_star_k, N_k=N_k, N_prime_k=N_prime_k, A_k=A_k, A_prime_k=A_prime_k, L_k=L_k, v=v, h=h, p=p, M_plus=M_plus, M_minus=M_minus, alpha=alpha, beta=beta, gamma=gamma, g=g, f=f, q=q, c=c, u=u, N_k_d = N_k_d, N_k_s = N_k_s, N_k_nod = N_k_nod, N_k_d_nod = N_k_d_nod, scarce = scarce, rho = rho)
    print('Expected value sample\n')
    x_star_EV_alpha_mu_hat, OF_EV_alpha_mu_hat, x_star_EEVS_alpha_mu_hat, OF_EEVS_alpha_mu_hat = EV_and_EEVS(alpha_mu_type=alpha_mu_hat, b = b, OMEGA = OMEGA, N=N, A=A, T=T, S=S, L=L, R=R, K=K, N_star_k=N_star_k, N_k=N_k, N_prime_k=N_prime_k, A_k=A_k, A_prime_k=A_prime_k, L_k=L_k, v=v, h=h, p=p, M_plus=M_plus, M_minus=M_minus, alpha=alpha, beta=beta, gamma=gamma, g=g, f=f, q=q, c=c, u=u, N_k_d = N_k_d, N_k_s = N_k_s, N_k_nod = N_k_nod, N_k_d_nod = N_k_d_nod, scarce = scarce, rho = rho)
    x_star_WS, OF_WS = WS(b = b, OMEGA = OMEGA, N=N, A=A, T=T, S=S, L=L, R=R, K=K, N_star_k=N_star_k, N_k=N_k, N_prime_k=N_prime_k, A_k=A_k, A_prime_k=A_prime_k, L_k=L_k, v=v, h=h, p=p, M_plus=M_plus, M_minus=M_minus, alpha=alpha, beta=beta, gamma=gamma, g=g, f=f, q=q, c=c, u=u, N_k_d = N_k_d, N_k_s = N_k_s, N_k_nod = N_k_nod, N_k_d_nod = N_k_d_nod, scarce = scarce, rho = rho)
    x_star_DE, OF_DE = DE(x_start = {}, b=b, OMEGA = OMEGA, N=N, A=A, T=T, S=S, L=L, R=R, K=K, N_star_k=N_star_k, N_k=N_k, N_prime_k=N_prime_k, A_k=A_k, A_prime_k=A_prime_k, L_k=L_k, v=v, h=h, p=p, M_plus=M_plus, M_minus=M_minus, alpha=alpha, beta=beta, gamma=gamma, g=g, f=f, q=q, c=c, u=u, N_k_d = N_k_d, N_k_s = N_k_s, N_k_nod = N_k_nod, N_k_d_nod = N_k_d_nod, scarce = scarce, rho = rho)


    print()
    print(f'OF_EEVS_population: {OF_EEVS_alpha_mu}')
    print()
    print(f'OF_EEVS_sample: {OF_EEVS_alpha_mu_hat}')
    print()
    print(f'OF_WS: {OF_WS}')
    print()
    print(f'OF_DE: {OF_DE}')
    print()
    print(f'Expectation of the Expected Value Solution (EVVS) with population mean: {OF_EEVS_alpha_mu}')
    print()
    print(f'Expectation of the Expected Value Solution (EVVS) with sample mean: {OF_EEVS_alpha_mu_hat}')
    print()
    print(f'Expected Value of Perfect Information (EVPI): {OF_WS-OF_DE}')
    print()
    print(f'Value of the Stochastic Solution (VSS) with population mean: {OF_DE-OF_EEVS_alpha_mu}')
    print(f'Value of the Stochastic Solution (VSS) with sample mean: {OF_DE-OF_EEVS_alpha_mu_hat}')
    print()

    with open(f"scarce_{scarce}_utility_type_{utility_type}_num_scenarios_{len(OMEGA)}_equal_probabilities_{same_probs}_nat_log_linear_pieces_length_{pieces_length}.txt", "w") as file:
        file.write(f"scarce_{scarce}_utility_type_{utility_type}_num_scenarios_{len(OMEGA)}_equal_probabilities_{same_probs}_nat_log_linear_pieces_length_{pieces_length}")
        file.write(f'\nOF_EEVS with population mean: {OF_EEVS_alpha_mu}')
        file.write('\n')
        file.write(f'\nOF_EEVS with sample mean: {OF_EEVS_alpha_mu_hat}')
        file.write('\n')
        file.write(f'OF_WS: {OF_WS}')
        file.write('\n')
        file.write(f'OF_DE: {OF_DE}')
        file.write('\n')
        file.write(f'Expectation of the Expected Value Solution (EEVS) with population mean: {OF_EEVS_alpha_mu}')
        file.write('\n')
        file.write(f'Expectation of the Expected Value Solution (EEVS) with sample mean: {OF_EEVS_alpha_mu_hat}')
        file.write('\n')
        file.write(f'Expected Value of Perfect Information (EVPI): {OF_WS-OF_DE}')
        file.write('\n')
        file.write(f'Value of the Stochastic Solution (VSS) with population mean: {OF_DE-OF_EEVS_alpha_mu}')
        file.write('\n')
        file.write(f'Value of the Stochastic Solution (VSS) with sample mean: {OF_DE-OF_EEVS_alpha_mu_hat}')

duration = 500# milliseconds
freq = 1000  # Hz
winsound.Beep(freq, duration)



scarce: 0.25


Expected value population

Expected value
Expectation of the expected value, scenario 1 out of 150
Expectation of the expected value, scenario 2 out of 150
Expectation of the expected value, scenario 3 out of 150
Expectation of the expected value, scenario 4 out of 150
Expectation of the expected value, scenario 5 out of 150
Expectation of the expected value, scenario 6 out of 150
Expectation of the expected value, scenario 7 out of 150
Expectation of the expected value, scenario 8 out of 150
Expectation of the expected value, scenario 9 out of 150
Expectation of the expected value, scenario 10 out of 150
Expectation of the expected value, scenario 11 out of 150
Expectation of the expected value, scenario 12 out of 150
Expectation of the expected value, scenario 13 out of 150
Expectation of the expected value, scenario 14 out of 150
Expectation of the expected value, scenario 15 out of 150
Expectation of the expected value, scenario 16 out of 150
Expectation of the expe